# GPT-4.1 mini
## Refined mutation operator generated by GPT-4.1 mini during Stage 2 of the proposed refinement framework.

In [ ]:
# imports 
import os
import numpy as np
import ioh 
import pandas as pd
import inspect
import importlib
import re 

In [ ]:
# First, we define the DE class in order to use different mutation strategies
class DE:
    """
    Class implementing the Differential Evolution (DE) algorithm in a
    generic manner, allowing the mutation strategy to be easily exchanged.
    """
    def __init__(self,
                 mutation_function,  # Mutation as a function
                 pop_size: int = 50,
                 num_iterations: int = 2000,
                 F: float = 0.8,  # typical value: 0.8
                 CR: float = 0.2):  # typical value: 0.2
        """
        Constructor of the DE algorithm.

        Args:
            mutation_function: Function implementing the mutation strategy.
            pop_size (int): Population size.
            num_iterations (int): Number of generations/iterations.
            F (float): Scaling factor for mutation.
            CR (float): Crossover rate (binomial crossover probability).
        """
        self.mutation_function = mutation_function
        self.pop_size = pop_size
        self.num_iterations = num_iterations
        self.F = F
        self.CR = CR

    def __call__(self, problem: ioh.problem.RealSingleObjective) -> None:
        dim = problem.meta_data.n_variables
        low_b, high_b = problem.bounds.lb[0], problem.bounds.ub[0]

        population = np.random.uniform(low_b, high_b, (self.pop_size, dim))
        fitness = np.array([problem(ind) for ind in population])

        for _ in range(self.num_iterations):
            for i in range(self.pop_size):
                # a. Mutation (now calling the external mutation function)
                mutant = self.mutation_function(
                    i=i,
                    population=population,
                    fitness=fitness,
                    F=self.F,
                    low_b=low_b,
                    high_b=high_b
                )

                # b. Crossover (Binomial) - unchanged
                trial_vector = np.copy(population[i])
                j_rand = np.random.randint(0, dim)
                crossover_mask = np.random.rand(dim) < self.CR
                crossover_mask[j_rand] = True
                trial_vector[crossover_mask] = mutant[crossover_mask]
                
                # c. Selection - unchanged
                trial_fitness = problem(trial_vector)
                
                if trial_fitness <= fitness[i]:
                    population[i] = trial_vector
                    fitness[i] = trial_fitness

In [ ]:
# Mutation functions
# All comments and descriptions within this function were generated
# by the corresponding LLM model during the refinement stage.
def mutation_refined_gpt41mini(i: int, population: np.ndarray, fitness: np.ndarray, F: float, low_b: float, high_b: float) -> np.ndarray:
    """
    Refined mutation based on 'DE/rand/1' with adaptive F.
    It introduces a difference term weighted by the inverse of the Pearson
    correlation between the selected vectors. Additionally, it incorporates
    an exponential attenuation factor on the jump magnitude to prevent
    excessively large steps, promoting more controlled and dynamic exploration.
    """
    pop_size = len(population)
    indices = list(range(pop_size))
    indices.remove(i)

    F_i = max(0.1, np.random.normal(loc=F, scale=0.1))

    r1, r2, r3 = np.random.choice(indices, 3, replace=False)

    diff1 = population[r1] - population[r2]
    diff2 = population[r2] - population[r3]

    epsilon = 1e-12
    diff1_centered = diff1 - np.mean(diff1)
    diff2_centered = diff2 - np.mean(diff2)
    numerator = np.sum(diff1_centered * diff2_centered)
    denominator = np.sqrt(np.sum(diff1_centered ** 2) * np.sum(diff2_centered ** 2)) + epsilon

    pearson_corr = numerator / denominator
    inv_corr_factor = 1 - abs(pearson_corr)

    diff_mean = (diff1 + diff2) / 2.0
    diff_scaled = diff_mean * (1 + inv_corr_factor)

    # Exponential attenuation based on the jump norm
    norm_diff = np.linalg.norm(diff_scaled)
    attenuation = np.exp(-norm_diff)
    diff_final = diff_scaled * attenuation

    mutant = population[r1] + F_i * diff_final

    return np.clip(mutant, low_b, high_b)

In [ ]:
def analyze_file_list(root_folder, file_list):
    all_results = []

    print(
        f"Analyzing {len(file_list)} files from folder: "
        f"'{os.path.abspath(root_folder)}'"
    )

    for file_name in file_list:
        # Build the full path to the file
        file_path = os.path.join(root_folder, file_name)
        
        try:
            # Extract information from the file name
            match_name = re.match(r"IOHprofiler_f(\d+)_(.*)\.info", file_name)
            if not match_name:
                continue
            
            func_id = int(match_name.group(1))
            func_name = match_name.group(2)

            # Read the file content
            with open(file_path, 'r') as f:
                content = f.read()
            
            # Extract results from repeated runs
            lines = content.splitlines()
            results_line = ""
            for i, current_line in enumerate(lines):
                if current_line.strip() == '%':
                    if i + 1 < len(lines):
                        results_line = lines[i + 1]
                        break
            
            if not results_line:
                raise ValueError(
                    "Results line not found after '%' delimiter"
                )

            matches_data = re.findall(
                r'(\d+)\s*:\s*(\d+)\s*\|\s*([\d.eE+-]+)',
                results_line
            )
            if not matches_data:
                raise ValueError("No result entries were found.")

            # Convert extracted data to numerical lists
            best_fitness_list = [float(m[2]) for m in matches_data]
            evaluations_list = [int(m[1]) for m in matches_data]
            
            # --- COMPUTATIONS ---
            # Compute mean, standard deviation, and median
            avg_fitness = np.mean(best_fitness_list)
            std_fitness = np.std(best_fitness_list)
            median_fitness = np.median(best_fitness_list)
            avg_evals = np.mean(evaluations_list)

            # --- STORE RESULTS ---
            all_results.append({
                "Function_ID": func_id,
                "Function_Name": func_name,
                "Avg_Fitness": avg_fitness,
                "Std_Fitness": std_fitness,
                "Median_Fitness": median_fitness,
                "Avg_Evals": avg_evals
            })
        except FileNotFoundError:
            print(f"- Warning: File '{file_path}' not found")
        except Exception as e:
            print(f"- Error processing file {file_name}: {e}")
    
    if not all_results:
        return None
        
    return pd.DataFrame(all_results)


In [ ]:

de_algorithm = DE(mutation_function=mutation_refined_gpt41mini, pop_size=50, F=0.8, CR=0.2)

experiment = ioh.Experiment(
    algorithm=de_algorithm,
    fids=range(1, 25), #Function range
    iids=[1], #Instances
    dims=[5], #Dimensions
    reps=30, #Number of Repetitions
    zip_output=False,
    folder_name="mutation_refined_gpt41mini",#Name of the output folder where results are stored
    old_logger=True
)

# To run the experiment, simply call it
experiment.run()

In [ ]:
# Analyze and display the results
if __name__ == "__main__":
    ROOT_FOLDER = "mutation_refined_gpt41mini" 

    files_to_analyze = [
        "IOHprofiler_f1_Sphere.info",
        "IOHprofiler_f2_Ellipsoid.info",
        "IOHprofiler_f3_Rastrigin.info",
        "IOHprofiler_f4_BuecheRastrigin.info",
        "IOHprofiler_f5_LinearSlope.info",
        "IOHprofiler_f6_AttractiveSector.info",
        "IOHprofiler_f7_StepEllipsoid.info",
        "IOHprofiler_f8_Rosenbrock.info",
        "IOHprofiler_f9_RosenbrockRotated.info",
        "IOHprofiler_f10_EllipsoidRotated.info",
        "IOHprofiler_f11_Discus.info",
        "IOHprofiler_f12_BentCigar.info",
        "IOHprofiler_f13_SharpRidge.info",
        "IOHprofiler_f14_DifferentPowers.info",
        "IOHprofiler_f15_RastriginRotated.info",
        "IOHprofiler_f16_Weierstrass.info",
        "IOHprofiler_f17_Schaffers10.info",
        "IOHprofiler_f18_Schaffers1000.info",
        "IOHprofiler_f19_GriewankRosenbrock.info",
        "IOHprofiler_f20_Schwefel.info",
        "IOHprofiler_f21_Gallagher101.info",
        "IOHprofiler_f22_Gallagher21.info",
        "IOHprofiler_f23_Katsuura.info",
        "IOHprofiler_f24_LunacekBiRastrigin.info"
    ]

    final_summary_df = analyze_file_list(root_folder=ROOT_FOLDER, file_list=files_to_analyze)

    print(final_summary_df)

In [ ]:
# Save the results to a .csv file
output_csv = "mutation_refined_gpt41mini.csv"
final_summary_df.to_csv(output_csv, index=False, encoding="utf-8-sig")
print(f"CSV file saved to: {output_csv}")